In [256]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import networkx as nx
import json
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from walker import load_a_HIN_from_pandas
from model import NSTrainSet, HIN2vec, train

In [222]:
df_train = pd.read_csv('/data/chaoyu/data553/yelp_train.csv')
df_val = pd.read_csv('/data/chaoyu/data553/yelp_val.csv')
df_comb = pd.concat([df_train, df_val])

In [225]:
df_business = pd.read_json('/data/chaoyu/data553/business.json', lines=True)
df_business

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,,1314 44 Avenue NE,Calgary,AB,T2E 6L6,51.091813,-114.031675,4.0,24,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Tours, Breweries, Pizza, Restaurants, Food, Ho...","{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'..."
1,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,,,Henderson,NV,89002,35.960734,-114.939821,4.5,3,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...","Chicken Wings, Burgers, Caterers, Street Vendo...","{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0..."
2,O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,Rosemont-La Petite-Patrie,1335 rue Beaubien E,Montréal,QC,H2G 1K7,45.540503,-73.599300,4.0,5,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","Breakfast & Brunch, Restaurants, French, Sandw...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
3,bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,,211 W Monroe St,Phoenix,AZ,85003,33.449999,-112.076979,1.5,8,1,None,"Insurance, Financial Services",None
4,8USyCYqpScwiNEb58Bt6CA,Action Engine,,2005 Alyth Place SE,Calgary,AB,T2H 0N5,51.035591,-114.027366,2.0,4,1,{'BusinessAcceptsCreditCards': 'True'},"Home & Garden, Nurseries & Gardening, Shopping...","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188588,sMQAZ3DkfrURFoJAyOhjEw,Ross Massage,,"8000 McKnight Rd, Ste 570",Pittsburgh,PA,15237,40.551152,-80.021213,2.5,9,0,"{'AcceptsInsurance': 'False', 'BusinessAccepts...","Skin Care, Beauty & Spas, Day Spas, Massage","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
188589,6hvuCibNS4uECetHb9MCQQ,Four Seasons Boutique,,3341 Babcock Blvd,Pittsburgh,PA,15237,40.534242,-80.019556,2.0,5,1,None,"Fashion, Women's Clothing, Accessories, Bridal...",None
188590,KleCXFYOmdACcQUvf6_XEg,Walmart Supercenter,,5825 Thunder Rd,Concord,NC,28027,35.378669,-80.724733,3.0,26,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Mobile Phones, Shopping, Department Stores, Fo...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
188591,3_fIsSxN2RBovQ_6EFtLzA,Residence Inn Charlotte Concord,,7601 Scott Padgett Pkwy,Concord,NC,28027,35.364366,-80.703454,4.0,19,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Event Planning & Services, Hotels & Travel, Ho...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [287]:
df_user = pd.read_json('/data/chaoyu/data553/user.json', lines=True)
df_user

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,lzlZwIpuSWXEnNS91wxjHw,Susan,1,2015-09-28,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1,XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,2015-09-05,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
2,QPT4Ud4H5sJVr68yXhoWFw,Andy,1,2016-07-21,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
3,i5YitlHZpf0B3R0s_8NVuw,Jonathan,19,2014-08-04,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
4,s4FoIXE_LSGviTHBe8dmcg,Shashank,3,2017-06-18,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518164,AwyHq4AtHU-Ds1khPUIIpQ,Stephanie,1,2018-03-02,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1518165,dZ-qwZX1_Ge-8KuSIJHhyA,Nicholas,4,2015-12-09,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1518166,Mv2-NyScx6SSvu2Swc5BEQ,Danielle,1,2014-12-20,None,0,0,0,1,None,...,0,0,0,0,0,0,0,0,0,0
1518167,9STgncH6CB8bXdcp2OjIuA,Eden,14,2015-02-21,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0


In [289]:
friends = set()
res = res.DataFrame(columns=['dest_node','source_node', 'weight', 'source_class', 'dest_class', 'edge_class'])
print(len(df_user))
for i in range(len(df_user)):
    if df_user['friends'][i] != 'None':
        friends_list = df_user['friends'][i].split(', ')
        #print(friends_list)
        for friend in friends_list:
            if friend in user_list:
                #print(i, tuple(sorted(list([df_user['user_id'][i], friend]))))
                #Detect replicate:
                friend_pair = tuple(sorted(list([df_user['user_id'][i], friend])))
                if friend_pair not in friends:
                    friends.add(friend_pair)
                    res.append({'dest_node': friend, 'source_node': df_user['user_id'][i], 'weight': 1, 'source_class': 'User', 'dest_class': 'User', 'edge_class': 'User-User'}, ignore_index=True)
#total 355182
print(len(friends))

1518169
995695


In [290]:
#list all categories
categories = set()
for cates in df_business['categories'].values:
    if cates == None:
        continue
    else:
        cates = cates.split(', ')
        for cate in cates:
            categories.add(cate)
categories = list(categories)
print(len(categories))
categories

1305


['& Probates',
 '3D Printing',
 'ATV Rentals/Tours',
 'Acai Bowls',
 'Accessories',
 'Accountants',
 'Acne Treatment',
 'Active Life',
 'Acupuncture',
 'Addiction Medicine',
 'Adoption Services',
 'Adult',
 'Adult Education',
 'Adult Entertainment',
 'Advertising',
 'Aerial Fitness',
 'Aerial Tours',
 'Aestheticians',
 'Afghan',
 'African',
 'Air Duct Cleaning',
 'Aircraft Dealers',
 'Aircraft Repairs',
 'Airlines',
 'Airport Lounges',
 'Airport Shuttles',
 'Airport Terminals',
 'Airports',
 'Airsoft',
 'Allergists',
 'Alternative Medicine',
 'Amateur Sports Teams',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Anesthesiologists',
 'Animal Assisted Therapy',
 'Animal Physical Therapy',
 'Animal Shelters',
 'Antiques',
 'Apartment Agents',
 'Apartments',
 'Appliances',
 'Appliances & Repair',
 'Appraisal Services',
 'Aquarium Services',
 'Aquariums',
 'Arabian',
 'Arcades',
 'Archery',
 'Architects',
 'Architectural Tours',
 'Argentine',
 'Armenian',
 'Art Classes'

In [291]:
lbe = LabelEncoder()
lbe.fit(categories)

LabelEncoder()

In [292]:
#Split categories and encoding (Lbe)
cates_split = []
for i in range(len(df_business)):
    cates_str = df_business['categories'][i]
    if cates_str != None and cates_str != 'None':
        cates = cates_str.split(', ')
        cates_lbe = lbe.transform(cates)
        cates_split.append(cates_lbe.tolist())
    else:
        cates_split.append([])
cates_split

[[1188, 167, 912, 1006, 466, 605],
 [239, 178, 229, 1121, 102, 472, 466, 1006, 419],
 [166, 1006, 477, 1026, 191],
 [630, 442],
 [571, 809, 1060, 706, 84, 393],
 [266, 466],
 [466, 96],
 [1006, 1176],
 [748, 1006],
 [463, 503, 1060],
 [1006, 647],
 [193, 1096, 1006],
 [105, 1109, 356, 178, 803, 1026, 1006],
 [1006, 846, 623, 749],
 [118, 103],
 [336, 1006, 1026],
 [803, 105, 33, 1155, 934, 1131, 1006, 1154],
 [1060, 433, 340],
 [442, 233, 1185],
 [33, 466, 96, 1006],
 [582, 723, 950, 289],
 [434, 178, 1006],
 [803, 105, 33, 1109, 1006],
 [466, 96, 266],
 [748, 1006],
 [748, 1006],
 [582, 573, 1040],
 [1006, 466, 197, 266],
 [823, 1060, 424, 549],
 [912, 1006],
 [803, 1006, 105, 239, 33],
 [1060, 490, 582, 809, 679, 571, 678],
 [434, 1006, 1026],
 [120, 1292, 644, 466, 33, 166, 1006],
 [549, 534, 1077, 531, 738, 310, 118],
 [1006, 191],
 [618, 466],
 [1037, 1006],
 [549, 904, 1008],
 [912, 434, 1006],
 [84, 74, 1060, 82, 212, 392],
 [266, 466, 166, 1006, 367],
 [582, 1060, 571, 664, 631

In [293]:
df_business.drop(['categories'], axis=1, inplace=True)
df_business.insert(0,'categories',cates_split)
df_business

,categories,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours
0,"[1188, 167, 912, 1006, 466, 605]",Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,,1314 44 Avenue NE,Calgary,AB,T2E 6L6,51.091813,-114.031675,4.0,24,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'..."
1,"[239, 178, 229, 1121, 102, 472, 466, 1006, 419]",AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,,,Henderson,NV,89002,35.960734,-114.939821,4.5,3,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...","{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0..."
2,"[166, 1006, 477, 1026, 191]",O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,Rosemont-La Petite-Patrie,1335 rue Beaubien E,Montréal,QC,H2G 1K7,45.540503,-73.599300,4.0,5,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
3,"[630, 442]",bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,,211 W Monroe St,Phoenix,AZ,85003,33.449999,-112.076979,1.5,8,1,None,None
4,"[571, 809, 1060, 706, 84, 393]",8USyCYqpScwiNEb58Bt6CA,Action Engine,,2005 Alyth Place SE,Calgary,AB,T2H 0N5,51.035591,-114.027366,2.0,4,1,{'BusinessAcceptsCreditCards': 'True'},"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188588,"[1077, 118, 332, 725]",sMQAZ3DkfrURFoJAyOhjEw,Ross Massage,,"8000 McKnight Rd, Ste 570",Pittsburgh,PA,15237,40.551152,-80.021213,2.5,9,0,"{'AcceptsInsurance': 'False', 'BusinessAccepts...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
188589,"[433, 1298, 4, 170, 1060]",6hvuCibNS4uECetHb9MCQQ,Four Seasons Boutique,,3341 Babcock Blvd,Pittsburgh,PA,15237,40.534242,-80.019556,2.0,5,1,None,None
188590,"[763, 1060, 340, 466, 433, 374, 392, 518, 354]",KleCXFYOmdACcQUvf6_XEg,Walmart Supercenter,,5825 Thunder Rd,Concord,NC,28027,35.378669,-80.724733,3.0,26,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
188591,"[419, 605, 604]",3_fIsSxN2RBovQ_6EFtLzA,Residence Inn Charlotte Concord,,7601 Scott Padgett Pkwy,Concord,NC,28027,35.364366,-80.703454,4.0,19,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [294]:
business_cate = set()
for i in range(len(df_business)): 
    for cate in df_business['categories'][i]:
        res.append({'dest_node': df_business['business_id'][i], 'source_node': cate, 'weight': 1, 'source_class': 'Cate', 'dest_class': 'Business', 'edge_class': 'Cate-Business'}, ignore_index=True)
        business_cate.add((df_business['business_id'][i], cate))
print(len(business_cate))

739013


In [295]:
city = set()
for ci in df_business.city.values:
    #print(ci)
    if ci == None:
        continue
    else:
        city.add(ci)
city = list(city)
print(len(city))
city

1111


['',
 '110 Las Vegas',
 'AGINCOURT',
 'Agincourt',
 'Ahwahtukee',
 'Ahwatukee',
 'Ahwatukee Foothills Village',
 'Airdrie',
 'Ajax',
 'Alberta Park Industrial',
 'Alburg',
 'Alburgh',
 'Allegheny',
 'Allison Park',
 'Ambridge',
 'Amherst',
 'Ange-Gardien',
 'Angus',
 'Anjou',
 'Ansnorveldt',
 'Anthem',
 'Argos',
 'Arnold',
 'Arrowhead',
 'Ashburn',
 'Aspinwall',
 'Auburn',
 'Auburn Township',
 'Auburn Twp',
 'Auerbach',
 'Aurora',
 'Austin',
 'Avalon',
 'Avon',
 'Avon Lake',
 'Avondale',
 "Baie-D'urfe",
 "Baie-d'Urfé",
 'Bainbridge',
 'Bainbridge Township',
 'Baldwin',
 'Ballantyne',
 'Balzac',
 'Banksville',
 'Bath',
 'Bay Village',
 'Bayview',
 'Beachwood',
 'Beaconsfield',
 'Beauharnois',
 'Bedford',
 'Bedford HTS',
 'Bedford Heights',
 'Bedford Hts.',
 'Beeton',
 'Bell',
 'Bellagio',
 'Belleville',
 'Bellevue',
 'Bellvue',
 'Belmont',
 'Beloeil',
 'Beltline',
 'Ben Avon',
 'Bentleyville',
 'Berea',
 'Berry',
 'Bethel Park',
 'Black Earth',
 'Blainville',
 'Blakeney',
 'Blawnox',
 '

In [296]:
lbe_city = LabelEncoder()
df_business['city'] = lbe_city.fit_transform(df_business['city'])
df_business

,categories,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours
0,"[1188, 167, 912, 1006, 466, 605]",Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,,1314 44 Avenue NE,122,AB,T2E 6L6,51.091813,-114.031675,4.0,24,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'..."
1,"[239, 178, 229, 1121, 102, 472, 466, 1006, 419]",AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,,,334,NV,89002,35.960734,-114.939821,4.5,3,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...","{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0..."
2,"[166, 1006, 477, 1026, 191]",O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,Rosemont-La Petite-Patrie,1335 rue Beaubien E,559,QC,H2G 1K7,45.540503,-73.599300,4.0,5,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
3,"[630, 442]",bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,,211 W Monroe St,706,AZ,85003,33.449999,-112.076979,1.5,8,1,None,None
4,"[571, 809, 1060, 706, 84, 393]",8USyCYqpScwiNEb58Bt6CA,Action Engine,,2005 Alyth Place SE,122,AB,T2H 0N5,51.035591,-114.027366,2.0,4,1,{'BusinessAcceptsCreditCards': 'True'},"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188588,"[1077, 118, 332, 725]",sMQAZ3DkfrURFoJAyOhjEw,Ross Massage,,"8000 McKnight Rd, Ste 570",726,PA,15237,40.551152,-80.021213,2.5,9,0,"{'AcceptsInsurance': 'False', 'BusinessAccepts...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
188589,"[433, 1298, 4, 170, 1060]",6hvuCibNS4uECetHb9MCQQ,Four Seasons Boutique,,3341 Babcock Blvd,726,PA,15237,40.534242,-80.019556,2.0,5,1,None,None
188590,"[763, 1060, 340, 466, 433, 374, 392, 518, 354]",KleCXFYOmdACcQUvf6_XEg,Walmart Supercenter,,5825 Thunder Rd,184,NC,28027,35.378669,-80.724733,3.0,26,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
188591,"[419, 605, 604]",3_fIsSxN2RBovQ_6EFtLzA,Residence Inn Charlotte Concord,,7601 Scott Padgett Pkwy,184,NC,28027,35.364366,-80.703454,4.0,19,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [297]:
business_city = set()
for i in range(len(df_business)):
    business_id = df_business['business_id'][i]
    city = 'c' + str(df_business['city'][i])
    res.append({'dest_node': df_business['business_id'][i], 'source_node': cate, 'weight': 1, 'source_class': 'City', 'dest_class': 'Business', 'edge_class': 'City-Business'}, ignore_index=True)
    business_city.add((business_id, city))
print(len(business_city))
business_city

188593


{('45SK-h0XA5ygJucqa03FPQ', 'c471'),
 ('k-3J66smBCnI30ZMwGpzAw', 'c985'),
 ('f1aiuS43bRV1sqGTMlLRWQ', 'c706'),
 ('sluWQx6hvnoDNwXY3Wr8Qw', 'c518'),
 ('oGefwIDMrFmSg9aBkv_T1Q', 'c559'),
 ('83rwReeaAul486BuYhJ0Dw', 'c176'),
 ('t4qlt6NheTTrdb3XhVv1Uw', 'c528'),
 ('DrZoYsaxLul52AX6fG-4Ew', 'c417'),
 ('ElBkIUL3zgzvjKylg5M-Cg', 'c518'),
 ('lCwUPM60Z4TTBKjASIOPsg', 'c428'),
 ('YQemT68K_jyoFlsLZLFjkQ', 'c307'),
 ('DE90wJCKRRkePq4KzGFUpA', 'c985'),
 ('bdNz9ySNCDCmaMTxlTKfuw', 'c592'),
 ('qIRqFlmvHDRe7kWBAsuVnQ', 'c417'),
 ('zWdO_P10pQlBKL95gYC_iw', 'c656'),
 ('anzCdD5eIEHwydTpXhLlUA', 'c176'),
 ('B3ucstFotOHce9VhSs9dog', 'c559'),
 ('1A6wgPpLZ6nW9ekRuBHDpA', 'c706'),
 ('QUk3AlQeDcoomTiPaXDjHw', 'c334'),
 ('UZYOo3nCn7FPUzV7TMVHNA', 'c35'),
 ('zSM5eouM0VcdfTC5kuT0nA', 'c706'),
 ('rKmWBixB_ucyrrTplX2Y3Q', 'c156'),
 ('Q9HFcCE_wYr0YDmoimUA9w', 'c307'),
 ('qd4qMzC2QrlgY4iDBMeLiQ', 'c985'),
 ('s_bx5mxoEYYn1-y8oE4JyQ', 'c985'),
 ('EbZO-25GoZqcxBStKA-CHQ', 'c417'),
 ('khtMC2-PQf_ylbRkWvnutw', 'c417'),
 (

In [298]:
business_pair = business_cate.union(business_city)
pairs = list(business_pair.union(friends))
print(len(business_pair))
print(len(pairs))

927606


In [299]:
df_business = df_business.rename(columns={'review_count': 'review_count_b'})
business_pre = pd.DataFrame(df_business[['business_id', 'city', 'state', 'is_open', 'stars', 'review_count_b', 'categories', 'postal_code']])
business_pre

,business_id,city,state,is_open,stars,review_count_b,categories,postal_code
0,Apn5Q_b6Nz61Tq4XzPdf9A,122,AB,1,4.0,24,"[1188, 167, 912, 1006, 466, 605]",T2E 6L6
1,AjEbIBw6ZFfln7ePHha9PA,334,NV,0,4.5,3,"[239, 178, 229, 1121, 102, 472, 466, 1006, 419]",89002
2,O8S5hYJ1SMc8fA4QBtVujA,559,QC,0,4.0,5,"[166, 1006, 477, 1026, 191]",H2G 1K7
3,bFzdJJ3wp3PZssNEsyU23g,706,AZ,1,1.5,8,"[630, 442]",85003
4,8USyCYqpScwiNEb58Bt6CA,122,AB,1,2.0,4,"[571, 809, 1060, 706, 84, 393]",T2H 0N5
...,...,...,...,...,...,...,...,...
188588,sMQAZ3DkfrURFoJAyOhjEw,726,PA,0,2.5,9,"[1077, 118, 332, 725]",15237
188589,6hvuCibNS4uECetHb9MCQQ,726,PA,1,2.0,5,"[433, 1298, 4, 170, 1060]",15237
188590,KleCXFYOmdACcQUvf6_XEg,184,NC,1,3.0,26,"[763, 1060, 340, 466, 433, 374, 392, 518, 354]",28027
188591,3_fIsSxN2RBovQ_6EFtLzA,184,NC,1,4.0,19,"[419, 605, 604]",28027


In [252]:
business_pre.to_csv('/data/chaoyu/data553/business_pre.csv', index=False)
res.to_csv('/data/chaoyu/data553/graph.csv', index=False)

In [40]:
user_name = pd.DataFrame(df_user['user_id'])
user_name.to_csv('~/user_name.csv', index=False)

In [41]:
user_name

,user_id
0,lzlZwIpuSWXEnNS91wxjHw
1,XvLBr-9smbI0m_a7dXtB7w
2,QPT4Ud4H5sJVr68yXhoWFw
3,i5YitlHZpf0B3R0s_8NVuw
4,s4FoIXE_LSGviTHBe8dmcg
...,...
1518164,AwyHq4AtHU-Ds1khPUIIpQ
1518165,dZ-qwZX1_Ge-8KuSIJHhyA
1518166,Mv2-NyScx6SSvu2Swc5BEQ
1518167,9STgncH6CB8bXdcp2OjIuA


In [42]:
business_name = pd.DataFrame(df_business['business_id'])
business_name.to_csv('~/business_name.csv', index=False)
business_name

,business_id
0,Apn5Q_b6Nz61Tq4XzPdf9A
1,AjEbIBw6ZFfln7ePHha9PA
2,O8S5hYJ1SMc8fA4QBtVujA
3,bFzdJJ3wp3PZssNEsyU23g
4,8USyCYqpScwiNEb58Bt6CA
...,...
188588,sMQAZ3DkfrURFoJAyOhjEw
188589,6hvuCibNS4uECetHb9MCQQ
188590,KleCXFYOmdACcQUvf6_XEg
188591,3_fIsSxN2RBovQ_6EFtLzA


In [253]:
# set method parameters
window = 4
walk = 10
walk_length = 300
embed_size = 128
neg = 5
sigmoid_reg = True 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device = {device}')

device = cuda:0


In [257]:
#Around 35 hours
demo_edge = pd.read_csv('./graph.csv', index_col=0)

edges = [demo_edge]

print('finish loading edges')

# init HIN
hin = load_a_HIN_from_pandas(edges)
hin.window = window

dataset = NSTrainSet(hin.sample(walk_length, walk), hin.node_size, neg=neg)

hin2vec = HIN2vec(hin.node_size, hin.path_size, embed_size, sigmoid_reg)

# load model
# hin2vec.load_state_dict(torch.load('hin2vec.pt'))

finish loading edges
load graph from edges...
finish loading graph!
init training dataset...
finished


In [ ]:
# set training parameters
# Around 14 hours on one 3090Ti
n_epoch = 10
batch_size = 256 #13GB GPU Memory
log_interval = 200

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
optimizer = optim.AdamW(hin2vec.parameters())
loss_function = nn.BCELoss()
hin2vec.to(device)
for epoch in range(n_epoch):
    train(log_interval, hin2vec, device, data_loader, optimizer, loss_function, epoch)

torch.save(hin2vec.state_dict(), 'hin2vec.pt')
# set output parameters [the output file is a bit different from the original code.]
node_vec_fname = 'emb_comb.csv'
path_vec_fname = None

embeddings = pd.DataFrame(hin2vec.start_embeds.weight.data.cpu().detach().numpy())

Train Epoch: 0 [196812800/196849800 (99.981%)]	Loss: 0.264		
Train Epoch: 1 [196812800/196849800 (99.981%)]	Loss: 0.660		
Train Epoch: 2 [196812800/196849800 (99.981%)]	Loss: 0.302		
Train Epoch: 3 [196812800/196849800 (99.981%)]	Loss: 0.260		
Train Epoch: 4 [196812800/196849800 (99.981%)]	Loss: 0.264		
Train Epoch: 5 [196812800/196849800 (99.981%)]	Loss: 0.698		
Train Epoch: 6 [196812800/196849800 (99.981%)]	Loss: 0.649		
Train Epoch: 7 [196812800/196849800 (99.981%)]	Loss: 0.652		
Train Epoch: 8 [196812800/196849800 (99.981%)]	Loss: 0.228		
Train Epoch: 9 [177152000/196849800 (89.993%)]	Loss: 0.280		

In [282]:
len(embeddings)

37354

In [283]:
df_emb = pd.DataFrame(pd.Series(embeddings), columns=['emb'])
df_emb = df_emb.reset_index().rename(columns={'index':'id'})
df_temp = df_emb['emb'].apply(pd.Series,index=[('emb_' + str(i)) for i in range(128)])
df_emb = df_emb.drop(['emb'], axis=1)
df_emb = pd.concat([df_emb, df_temp], axis=1)
df_emb

,id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,Kut9tVbDX_f7qKMRRoXg3w,0.319187,-0.230933,-0.350857,-0.170682,-0.035647,-0.273940,0.597013,0.550832,-0.132493,...,-0.429725,-0.200004,-0.440288,-0.749901,-0.066191,0.284120,-0.256287,-0.173324,-0.133117,0.480795
1,c189,0.216872,0.025564,-0.242560,0.097172,-0.195745,-0.143199,0.234812,-0.037678,-0.240529,...,0.120177,-0.232851,-0.386460,-0.247147,-0.544495,-0.156691,0.043667,-0.578423,0.123258,-0.129584
2,v8mNCaIvffBSJf4FG3uLMA,0.328663,-0.276356,-0.365657,-0.078980,-0.011250,-0.274186,0.587616,0.628540,-0.102939,...,-0.378291,-0.153889,-0.464437,-0.746305,-0.108673,0.255860,-0.229593,-0.219832,-0.195121,0.512699
3,216,0.184123,0.040301,-0.262876,0.098418,-0.212399,-0.097623,0.275428,-0.034213,-0.232194,...,0.154553,-0.242690,-0.349325,-0.212895,-0.514129,-0.167855,-0.018480,-0.553811,0.111801,-0.130182
4,bAU0v3UqPS20qMGwgI5rCg,0.245117,-0.011552,0.015350,-0.259858,-0.138364,-0.104209,0.011133,0.206763,-0.053244,...,0.067342,0.281099,-0.188487,-0.031987,-0.013796,0.197911,0.015002,-0.092728,-0.088925,0.137112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37349,T5vVvAqlZiKcPIXS857kQg,0.296756,-0.450935,-0.622456,0.176002,0.311683,0.197381,0.165500,-0.207518,-0.087405,...,0.139692,0.170923,0.075594,-0.204795,0.478822,0.372811,-0.597030,-0.237676,-0.285988,0.100081
37350,qpuYp31W89tq6vaGY6wLVg,-0.071674,-0.242618,-0.189458,0.190836,-0.556194,-0.470416,0.198477,0.049356,-0.253722,...,0.567403,-0.587527,0.075640,0.029325,-0.525911,-0.329693,-0.271512,-0.270256,-0.248347,-0.035122
37351,UkwuYrzwLXI_QidhVzUkgg,-0.004846,-0.712082,0.203882,-0.486615,0.165403,0.480705,0.365696,-0.313635,-0.228527,...,0.392680,-0.097421,0.241277,0.003852,0.048243,0.424685,1.025429,0.076182,-0.133716,-0.106642
37352,yLk3DFzlbzJYn0ACSQ8NAA,0.369633,0.009075,-0.306534,-0.003338,-0.101803,0.436942,0.714415,-0.029620,-0.012873,...,-0.018024,0.418485,-0.174428,-0.160364,-1.078631,-0.087058,0.392956,-0.027219,-0.614673,-0.097103


In [284]:
df_emb.to_csv('/data/chaoyu/data553/emb_comb.csv', index=False)